# APIを使ったデータのダウンロード

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install kaggle

In [1]:
import os
import json
f = open("/content/drive/MyDrive/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [3]:
!kaggle competitions download -c child-mind-institute-detect-sleep-states

 98% 795M/811M [00:04<00:00, 180MB/s]
100% 811M/811M [00:04<00:00, 177MB/s]


In [6]:
!unzip '/content/child-mind-institute-detect-sleep-states.zip'

Archive:  /content/child-mind-institute-detect-sleep-states.zip
  inflating: sample_submission.csv   
  inflating: test_series.parquet     
  inflating: train_events.csv        
  inflating: train_series.parquet    


# ここから実行

In [7]:
import sys
from pathlib  import Path

if 'google.colab' in sys.modules: #colab環境のファイルパス
  INPUT = ('/content/')
elif 'kaggle_web_client' in sys.modules: #kaggle環境のファイルパス
  INPUT = ('/kaggle/input/')

In [8]:
import pandas as pd

In [10]:
df = pd.read_csv(INPUT + 'train_events.csv')

In [26]:
df

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400
...,...,...,...,...,...
14503,fe90110788d2,33,wakeup,560604.0,2017-09-06T04:07:00-0400
14504,fe90110788d2,34,onset,574620.0,2017-09-06T23:35:00-0400
14505,fe90110788d2,34,wakeup,581604.0,2017-09-07T09:17:00-0400
14506,fe90110788d2,35,onset,NaN,NaN


In [12]:
from datetime import datetime
import pytz

# 文字列を作成
datetime_str = '2018-08-14T22:26:00-0400'

# datetimeオブジェクトに変換
datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S%z')

# UTCに変換する場合
datetime_utc = datetime_obj.astimezone(pytz.utc)

datetime_obj, datetime_utc


(datetime.datetime(2018, 8, 14, 22, 26, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),
 datetime.datetime(2018, 8, 15, 2, 26, tzinfo=<UTC>))

In [18]:
df.dtypes

series_id     object
night          int64
event         object
step         float64
timestamp     object
dtype: object

In [22]:
df.describe()

,night,step
count,14508.000000,9585.000000
mean,15.120072,214352.123944
std,10.286758,141268.408192
min,1.000000,936.000000
25%,7.000000,95436.000000
50%,14.000000,200604.000000
75%,21.000000,317520.000000
max,84.000000,739392.000000


In [28]:
for column in df.columns:
  print(len(df[column].unique()))
  print(df[column].unique())

277
['038441c925bb' '03d92c9f6f8a' '0402a003dae9' '04f547b8017d'
 '05e1944c3818' '062cae666e2a' '062dbd4c95e6' '08db4255286f'
 '0a96f4993bd7' '0cd1e3d0ed95' '0ce74d6d2106' '0cfc06c129cc'
 '0d0ad1e77851' '0dee4fda51c3' '0ec9fc461819' '0ef7d94fde99'
 '0f572d690310' '0f9e60a8e56d' '10469f6765bf' '1087d7b0ff2e'
 '10f8bc1f7b07' '12d01911d509' '1319a1935f48' '137771d19ca2'
 '137b99e936ab' '13b4d6a01d27' '148471991ffb' '154fe824ed87'
 '16fe2798ed0f' '1716cd4163b2' '1762ab70ec76' '188d4b7cd28b'
 '18a0ca03431d' '18b61dd5aae8' '1955d568d987' '1b92be89db4c'
 '1c7c0bad1263' '1d4569cbac0f' '1e6717d93c1d' '1f96b9668bdf'
 '207eded97727' '25e2b3dd9c3b' '2654a87be968' '27f09a6a858f'
 '280e08693c6d' '292a75c0b94e' '29c75c018220' '29d3469bd15d'
 '2b0a1fa8eba8' '2b8d87addea9' '2cd2340ca14d' '2e9ced2c7976'
 '2f7504d0f426' '2fbbee1a38e3' '2fc653ca75c7' '31011ade7c0a'
 '3318a0e3ed6f' '33ceeba8918a' '3452b878e596' '349c5562ee2c'
 '35826366dfc7' '361366da569e' '3664fe9233f9' '3665c86afaf5'
 '390b487231ce' '3a9

In [21]:
df.isnull().sum()

series_id       0
night           0
event           0
step         4923
timestamp    4923
dtype: int64

In [31]:
277 * 84

23268

In [38]:
df_datetime = df.copy()
df_datetime['timestamp'] = df_datetime['timestamp'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%dT%H:%M:%S%z')if x is not pd.isna(x) else None
)

ValueError: ignored